In [1]:
import tensorflow as tf

# Layers and models

Most models are made of layers. Layers are functions with a known mathematical structure that can be reused and have trainable variables. In TensorFlow, most high-level implementations of layers and models, such as Keras or Sonnet, are built on the same foundational class: tf.Module.

In [7]:
class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(5., name='train_me')
        self.non_trainable_variable = tf.Variable(5., trainable=False,
                                                   name='do_not_train_me')
    def __call__(self, x):
        return self.a_variable * x + self.non_trainable_variable

In [8]:
simple_module = SimpleModule(name='simple')

In [9]:
simple_module(tf.constant(5.))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>